In [18]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from environments import RodentSingleClipTrack
import numpy as np
import mediapy as media
import jax.numpy as jp

params = {
    "scale_factor": .9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 3,
    "clip_path": "12_22_1_250_clip_0.p",
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
    ],
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# env objects

In [19]:
env = RodentSingleClipTrack(params)

In [20]:
key = random.PRNGKey(0)
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)

check if non-jit reset and step work

In [12]:
n = env.reset(key)
n = env.step(n, random.uniform(key, shape=(env.sys.nu,)))

termination error: 0.023039329797029495


/home/charles/github/VNL-Brax-Imitation/mocap_preprocess.py:326: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  leaves = jax.tree_leaves(self)


jitted rollout:

In [21]:
next = jit_reset(key)
print("starting rollout")
mu = 0
sigma = .1
termination_errors = []
rollout = []
for _ in range(200):
    next = jit_step(next, mu + sigma * random.normal(key, shape=(env.sys.nu,)))
    rollout.append(next)
    termination_errors.append(next.info['termination_error'])

/home/charles/github/VNL-Brax-Imitation/mocap_preprocess.py:326: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  leaves = jax.tree_leaves(self)


starting rollout


/home/charles/github/VNL-Brax-Imitation/mocap_preprocess.py:326: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  leaves = jax.tree_leaves(self)
/home/charles/github/VNL-Brax-Imitation/mocap_preprocess.py:326: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  leaves = jax.tree_leaves(self)


[Array(0.02303933, dtype=float32),
 Array(0.06305969, dtype=float32),
 Array(0.08587934, dtype=float32),
 Array(0.09823309, dtype=float32),
 Array(0.10581864, dtype=float32),
 Array(0.10978494, dtype=float32),
 Array(0.11308961, dtype=float32),
 Array(0.11548334, dtype=float32),
 Array(0.11708232, dtype=float32),
 Array(0.11839269, dtype=float32),
 Array(0.12071632, dtype=float32),
 Array(0.1230047, dtype=float32),
 Array(0.1231153, dtype=float32),
 Array(0.12340453, dtype=float32),
 Array(0.1237914, dtype=float32),
 Array(0.12342937, dtype=float32),
 Array(0.12333444, dtype=float32),
 Array(0.12371013, dtype=float32),
 Array(0.12421259, dtype=float32),
 Array(0.12501422, dtype=float32),
 Array(0.12625234, dtype=float32),
 Array(0.1271095, dtype=float32),
 Array(0.127714, dtype=float32),
 Array(0.12824382, dtype=float32),
 Array(0.12870412, dtype=float32),
 Array(0.12911996, dtype=float32),
 Array(0.12900321, dtype=float32),
 Array(0.12923707, dtype=float32),
 Array(0.1295793, dtype=fl

render rollout:

In [22]:
import mediapy as media
import os

os.environ["MUJOCO_GL"] = "glfw"
rollout_data = [s.pipeline_state for s in rollout]

video = env.render(rollout_data, camera='close_profile', height=500, width=500)

media.show_video(video, fps=1.0 / env.dt)

This browser does not support the video tag.


test parallel envs:

In [26]:
env = RodentSingleClipTrack(params)
reset_fn = jax.jit(jax.vmap(env.reset))
step_fn = jax.jit(jax.vmap(env.step))

In [28]:
n_envs = 16
key_envs = jax.random.split(key, n_envs)
env_state = reset_fn(key_envs)

/home/charles/github/VNL-Brax-Imitation/mocap_preprocess.py:326: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  leaves = jax.tree_leaves(self)


In [29]:
env_state.info['cur_frame']

Array([41, 72, 54, 80, 56, 74, 38, 65, 69, 23, 20, 60, 55, 10, 89, 45],      dtype=int32)

In [30]:
# take a step in each env
mu = 0
sigma = 0.3
action = mu + sigma * random.normal(key, shape=(n_envs, env.sys.nu))
env_state = step_fn(env_state, action)

/home/charles/github/VNL-Brax-Imitation/mocap_preprocess.py:326: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  leaves = jax.tree_leaves(self)


In [32]:
print(f"cur_frame: {env_state.info['cur_frame']}")
print(f"termination_error: {env_state.info['termination_error']}")

cur_frame: [42 73 55 81 57 75 39 66 70 24 21 61 56 11 90 46]
termination_error: [0.02298566 0.02302505 0.0230299  0.02305004 0.02303933 0.02302711
 0.02305903 0.02303263 0.02303252 0.02312383 0.02312122 0.02303152
 0.02303508 0.02312163 0.02309475 0.02300531]


In [23]:
jax.tree_util.tree_map(
    lambda x: print(x.shape),
    env._ref_traj,
)

(250, 3)
(250, 5, 3)
(250, 18, 3)
(250, 18, 4)
(250, 3)
(250, 4, 3)
(250, 67)
(250, 67)
(0,)
(250, 3)
(250, 4)
(0,)
(250, 3)


ReferenceClip(angular_velocity=None, appendages=None, body_positions=None, body_quaternions=None, center_of_mass=None, end_effectors=None, joints=None, joints_velocity=None, markers=None, position=None, quaternion=None, scaling=None, velocity=None)